# Part 1: Deployment and Data Upload

In [1]:
#q1
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 39355482112 (36.65 GB)
DFS Remaining: 38824325120 (36.16 GB)
DFS Used: 531156992 (506.55 MB)
DFS Used%: 1.35%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.18.0.3:9866 (p4-dn-2.p4_default)
Hostname: 02f54e003a86
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 278929408 (266.01 MB)
Non DFS Used: 6113165312 (5.69 GB)
DFS Remaining: 19412180992 (18.08 GB)
DFS Used%: 1.08%
DFS Remaining%: 75.18%
Configured Cache Cap

In [2]:
import requests

url = " https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv"
r = requests.get(url, allow_redirects=True)

open('hdma-wi-2021.csv', 'wb').write(r.content)

174944099

In [3]:
!hdfs dfs -rm -f hdfs://boss:9000/single.csv
!hdfs dfs -rm -f hdfs://boss:9000/double.csv

Deleted hdfs://boss:9000/single.csv
Deleted hdfs://boss:9000/double.csv


In [4]:
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size=1048576 -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

In [5]:
!hdfs dfs -ls hdfs://boss:9000/

Found 2 items
-rw-r--r--   2 root supergroup  174944099 2023-10-26 05:29 hdfs://boss:9000/double.csv
-rw-r--r--   1 root supergroup  174944099 2023-10-26 05:29 hdfs://boss:9000/single.csv


In [6]:
#q2
!hdfs dfs -du -h hdfs://boss:9000/

166.8 M  333.7 M  hdfs://boss:9000/double.csv
166.8 M  166.8 M  hdfs://boss:9000/single.csv


# Part 2: WebHDFS

In [7]:
#q3
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
r.json()

{'FileStatus': {'accessTime': 1698298137987,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16392,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1698298141815,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [8]:
#q4
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset=0&noredirect=true")
r.json()["Location"]

'http://02f54e003a86:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [36]:
r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=GETFILEBLOCKLOCATIONS")
r.json()

{'BlockLocations': {'BlockLocation': [{'topologyPaths': ['/default-rack/172.18.0.3:9866'],
    'corrupt': False,
    'cachedHosts': [],
    'names': ['172.18.0.3:9866'],
    'offset': 0,
    'hosts': ['02f54e003a86'],
    'length': 1048576,
    'storageTypes': ['DISK']},
   {'topologyPaths': ['/default-rack/172.18.0.5:9866'],
    'corrupt': False,
    'cachedHosts': [],
    'names': ['172.18.0.5:9866'],
    'offset': 1048576,
    'hosts': ['ad1a53af8ffd'],
    'length': 1048576,
    'storageTypes': ['DISK']},
   {'topologyPaths': ['/default-rack/172.18.0.5:9866'],
    'corrupt': False,
    'cachedHosts': [],
    'names': ['172.18.0.5:9866'],
    'offset': 2097152,
    'hosts': ['ad1a53af8ffd'],
    'length': 1048576,
    'storageTypes': ['DISK']},
   {'topologyPaths': ['/default-rack/172.18.0.3:9866'],
    'corrupt': False,
    'cachedHosts': [],
    'names': ['172.18.0.3:9866'],
    'offset': 3145728,
    'hosts': ['02f54e003a86'],
    'length': 1048576,
    'storageTypes': ['DISK']},